In [34]:
import os
import warnings

import hdbscan
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import umap
from ax.plot.pareto_frontier import plot_pareto_frontier
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
from ax.service.ax_client import AxClient
from ax.service.managed_loop import optimize
from ax.service.utils.instantiation import ObjectiveProperties
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)

from ssl_wafermap.utilities.plotting import create_subplots

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_pickle("../data/interim/model_preds/SwaV_preds_full.pkl.xz")
cols = df.columns.difference(["failureType", "failureCode", "waferMap"])
data = df[cols].values

In [3]:
reducer = umap.UMAP(
    random_state=0,
    n_neighbors=30,
    min_dist=0,
    n_components=50,
    densmap=True,
    dens_lambda=0.1,
)
reduced_data = reducer.fit_transform(data)

UMAP(dens_lambda=0.2, densmap=True, min_dist=0, n_components=50, n_neighbors=30, random_state=0, verbose=True)
Thu May 18 17:18:58 2023 Construct fuzzy simplicial set
Thu May 18 17:18:58 2023 Finding Nearest Neighbors
Thu May 18 17:18:58 2023 Building RP forest with 15 trees
Thu May 18 17:18:59 2023 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	Stopping threshold met -- exiting after 3 iterations
Thu May 18 17:19:17 2023 Finished Nearest Neighbor Search
Thu May 18 17:19:22 2023 Construct embedding
Thu May 18 17:19:35 2023 Computing original densities


Epochs completed:   0%|            0/400 [00:00]

Thu May 18 17:23:54 2023 Finished embedding


In [35]:
def hdbscan_evaluation_function(parameterization):
    # Perform HDBSCAN clustering on the data with given parameters
    clusterer = hdbscan.HDBSCAN(
        min_samples=parameterization.get("min_samples"),
        min_cluster_size=parameterization.get("min_cluster_size"),
        cluster_selection_epsilon=parameterization.get("cluster_selection_epsilon"),
        metric=parameterization.get("metric"),
    )
    # clusterer.fit(data)
    clusterer.fit(reduced_data)

    # Calculate the number of clusters and points labeled as noise to constrain the optimization
    labels = clusterer.labels_
    n_clusters = labels.max() + 1
    n_noise = (labels == -1).sum()

    # Compute the silhouette score, Calinski-Harabasz score, and Davies-Bouldin score
    # These should be on the subset of the data NOT labeled as noise (i.e. labels != -1)
    subset_data, subset_labels = reduced_data[labels != -1], labels[labels != -1]
    silhouette = silhouette_score(subset_data, subset_labels)
    calinski_harabasz = calinski_harabasz_score(subset_data, subset_labels)
    davies_bouldin = davies_bouldin_score(subset_data, subset_labels)

    # Return the evaluation metrics and outcome constraints
    # These are tuples of the metrics with the SEM
    return {
        "n_noise": (n_noise, 0),
        "n_clusters": (n_clusters, 0),
        "silhouette": (silhouette, 0),
        "calinski_harabasz": (calinski_harabasz, 0),
        "davies_bouldin": (davies_bouldin, 0),
    }


# Create a search space for the hyperparameters
parameters = [
    {"name": "min_samples", "type": "range", "bounds": [1, 60], "value_type": "int"},
    {
        "name": "min_cluster_size",
        "type": "range",
        "bounds": [5, 100],
        "value_type": "int",
    },
    {
        "name": "cluster_selection_epsilon",
        "type": "range",
        "bounds": [0.1, 1.5],
        "value_type": "float",
    },
    {
        "name": "metric",
        "type": "choice",
        "values": ["euclidean", "manhattan", "canberra", "braycurtis"],
        "value_type": "str",
        "is_ordered": False,
    },
]


# Initialize the optimization client
ax_client = AxClient(random_seed=0)
ax_client.create_experiment(
    parameters=parameters,
    # Optimize cluster evaluation metrics while minimizing the number of noise points
    objectives={
        "silhouette": ObjectiveProperties(minimize=False),
        "calinski_harabasz": ObjectiveProperties(minimize=False),
        "davies_bouldin": ObjectiveProperties(minimize=True),
        "n_noise": ObjectiveProperties(minimize=True),
    },
    # We will constrain the number of possible clusters and points that are labeled as noise
    outcome_constraints=["n_clusters <= 30", "n_clusters >= 15"],
    overwrite_existing_experiment=True,
)

# Run 30 trials
for i in range(30):
    parameterization, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(
        trial_index=trial_index,
        raw_data=hdbscan_evaluation_function(parameterization),
    )

[INFO 05-18 19:38:37] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[WARNING 05-18 19:38:37] ax.service.ax_client: Random seed set to 0. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 05-18 19:38:37] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 05-18 19:38:37] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='min_samples', parameter_type=INT, range=[1, 60]), RangeParameter(name='min_cluster_size', parameter_type=INT, range=[5, 100]), Range

In [75]:
# Retrieve the best parameters
best_param_dict = ax_client.get_pareto_optimal_parameters()

# Display the best parameters
for summary in best_param_dict:
    params, (metrics, sems) = summary
    formatted_metrics = {key: f"{value:.3f}" for key, value in metrics.items()}
    print(f"Parameters: {params}")
    print(f"Metrics: {formatted_metrics}")
    print()

[INFO 05-18 21:56:01] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.
[INFO 05-18 21:56:01] ax.service.utils.best_point: Using inferred objective thresholds: [ObjectiveThreshold(calinski_harabasz >= 106369.25769861166), ObjectiveThreshold(davies_bouldin <= 0.41379343189545803), ObjectiveThreshold(silhouette >= 0.701476181620725)], as objective thresholds were not specified as part of the optimization configuration on the experiment.


Parameters: {'min_samples': 40, 'min_cluster_size': 100, 'cluster_selection_epsilon': 0.3784572732149105, 'metric': 'euclidean'}
Metrics: {'calinski_harabasz': '106388.464', 'davies_bouldin': '0.413', 'n_clusters': '15.822', 'n_noise': '2815.580', 'silhouette': '0.701'}

Parameters: {'min_samples': 40, 'min_cluster_size': 100, 'cluster_selection_epsilon': 0.37555919126415493, 'metric': 'euclidean'}
Metrics: {'calinski_harabasz': '106580.529', 'davies_bouldin': '0.414', 'n_clusters': '15.926', 'n_noise': '2879.327', 'silhouette': '0.701'}



```python
Parameters: {'min_samples': 40, 'min_cluster_size': 100, 'cluster_selection_epsilon': 0.3784572732149105, 'metric': 'euclidean'}
Metrics: {'calinski_harabasz': '106388.464', 'davies_bouldin': '0.413', 'n_clusters': '15.822', 'n_noise': '2815.580', 'silhouette': '0.701'}

Parameters: {'min_samples': 40, 'min_cluster_size': 100, 'cluster_selection_epsilon': 0.37555919126415493, 'metric': 'euclidean'}
Metrics: {'calinski_harabasz': '106580.529', 'davies_bouldin': '0.414', 'n_clusters': '15.926', 'n_noise': '2879.327', 'silhouette': '0.701'}
```

In [83]:
# In this case there are two optimal sets of hyperparameters that are fairly similar
# We will just use the first one
best_params, (best_metrics, best_sems) = next(iter(best_param_dict.values()))

# Perform HDBSCAN clustering on the data with the best parameters
clusterer = hdbscan.HDBSCAN(**best_params)
clusterer.fit(reduced_data)

HDBSCAN(cluster_selection_epsilon=0.3784572732149105, min_cluster_size=100,
        min_samples=40)

In [101]:
# Create a 2D UMAP embedding of the data
reducer_2d = umap.UMAP(random_state=0)
embedding_2d = reducer_2d.fit_transform(data)

emb_df = pd.DataFrame(embedding_2d, columns=["x", "y"])
emb_df["failureType"] = df["failureType"].values
emb_df["waferMap"] = df["waferMap"].values
emb_df["cluster"] = pd.Series(clusterer.labels_).astype("category").values
emb_df.sort_values("cluster", inplace=True)

In [132]:
fig = px.scatter(
    emb_df,
    x="x",
    y="y",
    color="cluster",
    color_discrete_sequence=sns.color_palette(
        "tab20", n_colors=clusterer.labels_.max() + 1
    ).as_hex(),
    width=800,
    height=600,
    template="simple_white",
    color_discrete_map={-1: "lightgray"},
)
# fig.update_traces(
#     marker=dict(
#         line=dict(
#             width=emb_df["cluster"].map(lambda x: 0 if x == -1 else 0.5).values,
#             color="white",
#         )
#     )
# )
fig.show()